<a href="https://colab.research.google.com/github/gzholtkevych/LambdaStudy/blob/main/syntax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1><b>Syntax of Lambda Calculus</b></H1>

# Import of the needed external program facilities

In [1]:
from typing import Dict, Optional, Tuple, Union, List
from typing_extensions import Self

# Variables

## Understanding

A variable is an atomic entity of a lambda calculus that refers to itself.<br/>
In other words,

>the value of a variable is its name.

We understand **atomicity** as

>the property of a one-to-one correspondence between variables and memory blocks allocated for them.

## Implementing

The notion is of a var implemented by the class `var`.

*Static fields*

```
__declared       is the dictionary of all declared variables;
                 the key of a variable in this dictionary is the variable name
__check_varname  is a function that takes a string representing a variable name and
                 throws an exception if the string does not match the variable
                 name format namely, an identifier starting with a lowercase letter
```

*Class methods*

```
get           returns either the variable by its name if that variable is already declared or
              returns `None` if it is not declared
all_declared  returns the list of all declared variables
```

In [2]:
class var(str):

    __declared: Dict[str, Self] = {}  # the dictionary of declared variables
                                     # with the variable name as its key

    def __check_varname(varname: str) -> None:
        if not isinstance(varname, str):
            raise TypeError(
                "Invalid variable name type. Type 'str' is expected.")
        if not (varname.isidentifier() and varname[0].islower()):
            raise ValueError(
                "Invalid variable name value. An identifier that starts with "
                "a lowercase letter, is expected.")

    def __new__(cls, varname: str) -> Self:
        var.__check_varname(varname)
        if varname not in cls.__declared:  # 'varname' is a new name
            cls.__declared[varname] = str.__new__(cls, varname)
        return cls.__declared[varname]

    def __eq__(self, another: Self) -> bool:
        if not isinstance(another, var):
            return False
        return super().__eq__(another)

    @classmethod
    def get(cls, varname: str) -> Optional[Self]:
        cls.__check_varname(varname)
        try:
            return cls.__declared[varname]
        except KeyError:
            return None

    @classmethod
    def all_declared(cls) -> List[Self]:
        return [cls.__declared[key] for key in cls.__declared]

# Lambda Terms

## Understanding

Lambda terms are built from variables using three constructors

- `atom`, which coerces a variable to the corresponding atomic term;
- `application`, which applies one term to another;
- `abstraction`, which declares one variable of the term as a function argument.

The following notation is usually used.

- **atom:** $\dfrac{x\text{ is a variable}}{x\text{ is a term}}$;
- **application:** $\dfrac{M\text{ is a term}\quad N\text{ is a term}}{(MN)\text{ is a term}}$;
- **abstraction:** $\dfrac{x\text{ is a variable}\quad M\text{ is a term}}{(\lambda\,x\mathop{.}M)\text{ is a term}}$.

The symbols '$\lambda$', '(', ')', and '.' are punctuation marks Lambda Calculus and they do not belong to the set of variables.

## Implementing

In [39]:
class Term(tuple):

    def __new__(cls, *args: Tuple) -> Self:
        if not args:
            raise ValueError("missing constructor argument(s)")
        if args[0] == 0:  # the case of an atom
            if len(args) != 2:
                raise ValueError("incorrect constructor argument number")
            if not isinstance(args[1], var):
                raise TypeError("incorrect constructor argument type")
        elif args[0] == 1:  # the case of an application
            if len(args) != 3:
                raise ValueError("incorrect constructor argument number")
            if not all(isinstance(arg, Term) for arg in args[1 :]):
                raise TypeError("incorrect constructor argument(s) type")
        elif args[0] == 2:  # the case of an abstraction
            if len(args) != 3:
                raise ValueError("incorrect constructor argument number")
            if not (isinstance(args[1], var) and isinstance(args[2], Term)):
                raise TypeError("incorrect constructor argument(s) type")
        else:
            raise ValueError("unrecognized term constructor")
        return super().__new__(cls, args)

    @classmethod
    def atm(cls, x: Union[str, var]) -> Self:
        if isinstance(x, var):
            return cls(0, x)
        if isinstance(x, str):
            return cls(0, var(x))
        raise TypeError("invalid argument type")

    @classmethod
    def app(cls, t1: Self, t2: Self) -> Self:
        return cls(1, t1, t2)

    @classmethod
    def abs(cls, x: Union[str, var], t: Self) -> Self:
        if isinstance(x, var):
            return cls(2, x, t)
        if isinstance(x, str):
            return cls(2, var(x), t)

    def __str__(self):
        if self[0] == 0:
            return self[1]
        if self[0] == 1:
            return f"({self[1]} {self[2]})"
        # self[0] == 2
        return f"(λ {self[1]}.{self[2]})"